<a href="https://colab.research.google.com/github/Ahnkyuwon504/AI-modeling/blob/main/instruction-tuning/gemma_ko_2b_instruction_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 환경 설정

## 1-1. 필수 라이브러리 설치

In [1]:
!pip3 install -q -U transformers==4.38.2
!pip3 install -q -U datasets==2.18.0
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.9.0
!pip3 install -q -U trl==0.7.11
!pip3 install -q -U accelerate==0.27.2

## 1-2. Import Modules

In [2]:
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

## 1-3. HugggingFace

In [3]:
from huggingface_hub import notebook_login
notebook_login()

# 2. DataSet

## 2-1. 데이터 로드

In [6]:
from datasets import load_dataset
dataset = load_dataset("beomi/KoAlpaca-v1.1a")

## 2-2. 데이터 탐색

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'url'],
        num_rows: 21155
    })
})

## 2-3. 데이터셋 예시

In [8]:
dataset['train'][0]

{'instruction': '양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?',
 'output': '양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\n\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n\n고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.',
 'url': 'https://kin.naver.com/qna/detail.naver?d1id=11&dirId=1116&docId=55320268'}

# 3. Pre-trained Model(Gemma base)

## 3-1. 모델 로드

`device_map` 파라미터를 통해 모델을 로드할 디바이스를 지정한다.

`{"":0}`은 모델의 모든 파라미터를 첫 번째 GPU(즉, GPU 0)에 로드하라는 의미

`add_special_tokens` 파라미터는 특별 토큰(special tokens)을 추가할지 여부를 결정한다. 특별 토큰은 모델에 따라 다르지만, 일반적으로 문장의 시작과 끝을 나타냄.

In [7]:
PRE_TRAINED_MODEL = "beomi/gemma-ko-2b"

model = AutoModelForCausalLM.from_pretrained(PRE_TRAINED_MODEL, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL, add_special_tokens=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

## 3-2. 추론 수행

In [8]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

In [9]:
prompt = "창원시에 대해 알려줘"


`do_sample` 파라미터를 통해 텍스트를 생성할 때 확률적 샘플링을 사용할지 여부를 결정합니다. True로 설정하면 확률에 기반하여 토큰을 샘플링하여 다양한 출력을 생성할 수 있습니다. False로 설정하면 그리디 서치(가장 높은 확률의 토큰을 선택)를 사용합니다.

`temperature` 파라미터를 통해 샘플링 시 확률 분포를 제어하는 매개변수입니다. 값이 낮을수록 확률 분포가 더욱 집중되며, 모델이 더 보수적으로 예측하게 됩니다. 값이 높을수록 확률 분포가 넓어져 더 다양하고 창의적인 출력을 생성할 수 있습니다.
온도가 낮을수록  보수적이고 일관된 출력을 생성하며, 온도가 높을수록 더 다양한 출력을 생성할 가능성이 높습니다.

`top_k` 매 샘플링 단계에서 고려할 상위 k개의 토큰만 사용합니다. 상위 50개의 후보 토큰 중에서만 선택하여 다음 토큰을 생성합니다. 희귀하고 덜 관련된 토큰을 제외하고, 상위 후보 중에서 샘플링하여 더 합리적인 출력을 생성하도록 합니다.

`top_p` 누적 확률이 p(0.95) 이상이 되는 상위 토큰 집합에서 샘플링합니다. 이 방법을 Nucleus Sampling이라고도 합니다. 확률 질량의 95%를 차지하는 상위 토큰을 선택하여 다음 토큰을 생성합니다. 이로 인해 유연한 샘플링이 가능하며, 확률이 매우 낮은 토큰을 배제하여 출력을 향상시킵니다.

`repetition_penalty` 반복 페널티를 적용하여 동일한 단어가 반복되지 않도록 합니다. 1보다 큰 값을 사용하면 반복되는 단어에 패널티를 부여합니다. 출력을 더 다양하게 만들어 반복을 방지합니다. 값이 클수록 더 강한 패널티가 적용됩니다.

`add_special_tokens` 특별 토큰(special tokens)을 추가할지 여부를 결정합니다. 예를 들어, 문장의 시작과 끝을 나타내는 토큰 등을 추가합니다.

In [10]:
outputs = pipe(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    repetition_penalty=1.2,
    add_special_tokens=True
)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [11]:
outputs[0]

{'generated_text': '창원시에 대해 알려줘요. 2018년 4월'}

# 4. Instruction Tuning

In [9]:
!nvidia-smi

Sat Jun 15 07:20:45 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## 4-1. 학습 프롬프트 생성

In [11]:
train_data = dataset['train']
train_data

Dataset({
    features: ['instruction', 'output', 'url'],
    num_rows: 21155
})

In [12]:
train_data[0]

{'instruction': '양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?',
 'output': '양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\n\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n\n고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.',
 'url': 'https://kin.naver.com/qna/detail.naver?d1id=11&dirId=1116&docId=55320268'}

In [13]:
def generate_prompt(example):
    output_texts = []
    for i in range(len(example['instruction'])):
        prompt = f"### Instruction: {example['instruction'][i]}\n\n### Response: {example['output'][i]}<eos>"
        output_texts.append(prompt)
    return output_texts

In [20]:
print(generate_prompt(train_data[:1])[0])

### Instruction: 양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?

### Response: 양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. 

식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.

 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? 

고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.<eos>


## 4-2. QLoRA 설정

`quantization_config` 파라미터를 통해 모델 양자화(quantization) 설정을 지정한다. bnb_config는 양자화 설정이 담긴 변수로, 모델의 메모리 사용량을 줄이고 성능을 최적화하는 데 사용됨.

In [21]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

BASE_MODEL = "beomi/gemma-ko-2b"
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map={"":0}, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, add_special_tokens=True)
tokenizer.padding_side = 'right'

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

`LoRA (Low-Rank Adaptation)` 설정을 통해 자연어 처리 모델의 파인튜닝을 위해 사용. 특정 파라미터에 대해 저차원 근사화를 사용하여 메모리와 계산 효율성을 높이는 기법

In [22]:
lora_config = LoraConfig(
    r=6,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

## 4-3. Trainer

`max_seq_length` 모델이 처리할 수 있는 입력 시퀀스의 최대 길이로 입력 시퀀스가 이 길이를 초과하면 잘리거나 패딩됨

`peft_config=lora_config` LoRA 설정을 지정

`formatting_func=generate_prompt` 입력 데이터를 모델의 입력 형식에 맞게 변환하는 함수로 프롬프트를 생성하는 데 사용됩니다.


`args=TrainingArguments(...)` 학습 설정을 위한 인자들을 포함하는 TrainingArguments 객체

+ `output_dir="outputs"` 학습 결과(모델, 로그 등)를 저장할 디렉토리입니다.

+ `max_steps=3000` 최대 학습 스텝 수

`per_device_train_batch_size=1` 각 디바이스(예: GPU)에서 학습할 배치 크기

`gradient_accumulation_steps=4` 그래디언트 누적 스텝 수로 4 스텝 동안 그래디언트를 누적하여 업데이트를 수행

`optim="paged_adamw_8bit"` 옵티마이저로 paged_adamw_8bit를 사용합니다. 이는 메모리 효율성을 높이기 위해 8비트 정밀도를 사용하는 AdamW 옵티마이저

`warmup_steps=0.03` 학습 초기의 워밍업 스텝 비율입니다. 전체 스텝의 3% 동안 학습률을 점진적으로 증가

`learning_rate=2e-4` 학습률입니다. 학습 속도를 제어하는 중요한 하이퍼파라미터로, 0.0002로 설정되어 있습니다.

`fp16=True` 16비트 부동 소수점(FP16) 혼합 정밀도 학습을 사용합니다. 이를 통해 메모리 사용량을 줄이고 학습 속도를 높일 수 있습니다.

`logging_steps=100` 로그를 기록할 스텝 간격입니다. 매 100 스텝마다 학습 로그를 기록합니다.

`push_to_hub=False` 학습된 모델을 Hugging Face Hub에 업로드하지 않습니다.

`report_to='none'` 로그를 기록할 때 사용할 툴입니다. 여기서는 로그를 외부 툴로 보고하지 않습니다.



In [23]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    max_seq_length=512,
    args=TrainingArguments(
        output_dir="outputs",
#        num_train_epochs = 1,
        max_steps=3000,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_8bit",
        warmup_steps=0.03,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=100,
        push_to_hub=False,
        report_to='none',
    ),
    peft_config=lora_config,
    formatting_func=generate_prompt,
)

Map:   0%|          | 0/21155 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:294: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [24]:
trainer.train()

Step,Training Loss
100,1.749100
200,1.669700
300,1.640700
400,1.637900
500,1.625800
600,1.632800
700,1.650300
800,1.635900
900,1.635800
1000,1.624300


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in

TrainOutput(global_step=3000, training_loss=1.6289947153727213, metrics={'train_runtime': 6009.9094, 'train_samples_per_second': 1.997, 'train_steps_per_second': 0.499, 'total_flos': 4.178911278637056e+16, 'train_loss': 1.6289947153727213, 'epoch': 0.57})

4-4. Finetuned Model 저장

In [25]:
ADAPTER_MODEL = "loar_adapt_it"

trainer.model.save_pretrained(ADAPTER_MODEL)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [27]:
!ls -alh loar_adapt_it

total 29M
drwxr-xr-x 2 root root 4.0K Jun 15 09:21 .
drwxr-xr-x 1 root root 4.0K Jun 15 09:21 ..
-rw-r--r-- 1 root root  688 Jun 15 09:21 adapter_config.json
-rw-r--r-- 1 root root  29M Jun 15 09:21 adapter_model.safetensors
-rw-r--r-- 1 root root 5.0K Jun 15 09:21 README.md


gemma-ko-2b 모델과 합쳐 하나의 finetuned 모델로

In [29]:
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map='auto', torch_dtype=torch.float16)
model = PeftModel.from_pretrained(model, ADAPTER_MODEL, device_map='auto', torch_dtype=torch.float16)

model = model.merge_and_unload()
model.save_pretrained('gemma-ko-2b-it')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [30]:
!ls -alh gemma-ko-2b-it

total 4.7G
drwxr-xr-x 2 root root 4.0K Jun 15 09:22 .
drwxr-xr-x 1 root root 4.0K Jun 15 09:21 ..
-rw-r--r-- 1 root root  637 Jun 15 09:21 config.json
-rw-r--r-- 1 root root  132 Jun 15 09:21 generation_config.json
-rw-r--r-- 1 root root 4.7G Jun 15 09:22 model-00001-of-00002.safetensors
-rw-r--r-- 1 root root  65M Jun 15 09:22 model-00002-of-00002.safetensors
-rw-r--r-- 1 root root  14K Jun 15 09:22 model.safetensors.index.json


# 5. Finetuning 모델 추론

In [ ]:
!nvidia-smi

## 5-1. 모델 로드

In [4]:
BASE_MODEL = "beomi/gemma-ko-2b"
FINETUNE_MODEL = "./gemma-ko-2b-it"

finetune_model = AutoModelForCausalLM.from_pretrained(FINETUNE_MODEL, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, add_special_tokens=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## 5-2. 모델 추론

In [5]:
pipe_finetuned = pipeline(
    "text-generation",
    model=finetune_model,
    tokenizer=tokenizer,
    max_new_tokens=512
    )

중요. 프롬프트를 학습시킨 데이터셋과 동일한 포맷으로 작성

In [7]:
prompt = "창원특별시에 대해 알려줘"
formatted_prompt = f"### Response: {prompt}\n\n### Response:"

In [8]:
outputs = pipe_finetuned(
    formatted_prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    repetition_penalty=1.2,
    add_special_tokens=True
)
print(outputs[0]["generated_text"][len(formatted_prompt):])

 창원은 경상남도의 중심지로, 1970년대부터 개발되어 현재는 대규모 도시가 되어있습니다. 창원에는 다양한 문화와 관광명소들이 있으며, 특히 마산합포구에서는 해양스포츠를 즐길 수 있는 곳으로 유명합니다. 또한, 창원을 방문하면서 주요 명소들을 살펴보면, 미륵사지, 고성동굴 등이 있습니다.
